In [2]:
import numpy as np 
import seaborn as sns 
import pandas as pd 
import ml_utils as mt 

In [4]:
ld_train=pd.read_csv("./loan_data_train.csv")

In [6]:
def dtr(orig_col):
    
    mod_col=orig_col.str.replace('%','')
    mod_col=pd.to_numeric(mod_col,errors='coerce')
    
    return mod_col


def fico(orig_col):
    k=orig_col.str.split('-',expand=True)
    
    for i in [0,1]:
        k[i]=pd.to_numeric(k[i],errors='coerce')
    
    mod_col=0.5*(k[0]+k[1])
    
    return mod_col


def el(orig_col):
    
    inter_col=orig_col.str.replace('10+ years','10',regex=False)
    inter_col=inter_col.str.replace('< 1 year','0',regex=False)
    inter_col=inter_col.str.replace('years','').str.replace('year','')
    
    mod_col=pd.to_numeric(inter_col,errors='coerce')
    
    return mod_col


In [11]:
cat_to_dummies_cols=['Loan.Length','Loan.Purpose','State','Home.Ownership']
# groups of columns for which we need to create dummies 
cat_to_num_cols=['Amount.Requested',#amount funded by the investor
                 'Open.CREDIT.Lines','Revolving.CREDIT.Balance']
# columns which contain numeric info but only require simple typecasting
simple_num_cols=['Monthly.Income','Inquiries.in.the.Last.6.Months']
# columns which are all ready numeric and contain numeric information
custom_func_dict_cols={'Debt.To.Income.Ratio':dtr,'FICO.Range':fico,'Employment.Length':el}
# columns which require custom processing to extract numeric information from the data

In [12]:
ld_pipe=mt.DataPipe(cat_to_dummies=cat_to_dummies_cols,
                 cat_to_num=cat_to_num_cols,
                 simple_num=simple_num_cols,
                 custom_func_dict=custom_func_dict_cols)


In [13]:
ld_pipe.fit(ld_train)

d:\my_codes_26_12_24_1\upes_repo\UPES_Courses_through_Notes_and_Codes\ML-DL-Intensive_Training\Day-05-29-1-25_\ml_utils.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[col]=pd.to_numeric(x[col],errors='coerce')
d:\my_codes_26_12_24_1\upes_repo\UPES_Courses_through_Notes_and_Codes\ML-DL-Intensive_Training\Day-05-29-1-25_\ml_utils.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[col]=self.col_func_dict[col](x[col])
d:\my_codes_26_12_24_1\upes_repo\UPES_Courses_through_Notes_and_Codes\ML-DL-Inten

In [14]:
x_train=ld_pipe.transform(ld_train)

d:\my_codes_26_12_24_1\upes_repo\UPES_Courses_through_Notes_and_Codes\ML-DL-Intensive_Training\Day-05-29-1-25_\ml_utils.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[col]=pd.to_numeric(x[col],errors='coerce')
d:\my_codes_26_12_24_1\upes_repo\UPES_Courses_through_Notes_and_Codes\ML-DL-Intensive_Training\Day-05-29-1-25_\ml_utils.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[col]=self.col_func_dict[col](x[col])
d:\my_codes_26_12_24_1\upes_repo\UPES_Courses_through_Notes_and_Codes\ML-DL-Inten

In [18]:
y_train=ld_train['Interest.Rate'].str.replace('%','').astype(float)
y_train

0       18.49
1       17.27
2       14.33
3       16.29
4       12.23
        ...  
2195    23.28
2196    14.33
2197    15.31
2198    20.99
2199    10.16
Name: Interest.Rate, Length: 2200, dtype: float64

In [19]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

In [20]:
lr = LinearRegression()

In [21]:
scores = cross_val_score(lr, x_train, y_train, cv=10, scoring='neg_mean_absolute_error')

In [23]:
scores.mean()

-1.6139479356558968

In [24]:
lr.fit (x_train, y_train)

LinearRegression()

In [25]:
lr.intercept_

74.23374119067523

In [27]:
list(zip(x_train.columns,lr.coef_))

[('Loan.Length_36 months', 1.2928421354433781),
 ('Loan.Length_60 months', 4.459656083490472),
 ('Loan.Purpose_debt_consolidation', -1.4966786385374202),
 ('Loan.Purpose_credit_card', -1.6131316101962816),
 ('Loan.Purpose_other', -0.6806048195811828),
 ('Loan.Purpose_home_improvement', -1.4168464814314938),
 ('Loan.Purpose_major_purchase', -1.142536623365205),
 ('Loan.Purpose_small_business', -1.0027999343050427),
 ('Loan.Purpose___other__', -0.7324431318239972),
 ('Loan.Purpose_car', -1.24648883543951),
 ('Loan.Purpose_wedding', -1.7238229254752255),
 ('Loan.Purpose_medical', -1.3295157484774083),
 ('State_CA', -0.21833644643734693),
 ('State_NY', -0.22362942183483256),
 ('State___other__', -0.014458793415074526),
 ('State_FL', -0.0879237034698414),
 ('State_TX', 0.42047286413838697),
 ('State_PA', -0.5173699920557517),
 ('State_IL', -0.6160818675429932),
 ('State_GA', -0.2512258403694186),
 ('State_NJ', -0.4167415970533856),
 ('State_VA', -0.1531211629709439),
 ('State_MA', -0.121445

In [30]:
effect_df = x_train*lr.coef_
effect_df.mean()

Loan.Length_36 months               1.011943
Loan.Length_60 months               0.964907
Loan.Purpose_debt_consolidation    -0.780314
Loan.Purpose_credit_card           -0.288897
Loan.Purpose_other                 -0.053830
Loan.Purpose_home_improvement      -0.086943
Loan.Purpose_major_purchase        -0.043624
Loan.Purpose_small_business        -0.036465
Loan.Purpose___other__             -0.018311
Loan.Purpose_car                   -0.025496
Loan.Purpose_wedding               -0.027424
Loan.Purpose_medical               -0.015712
State_CA                           -0.037316
State_NY                           -0.023481
State___other__                    -0.001433
State_FL                           -0.005955
State_TX                            0.027904
State_PA                           -0.020695
State_IL                           -0.024363
State_GA                           -0.009364
State_NJ                           -0.015344
State_VA                           -0.004872
State_MA  